In [11]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None


# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [21]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "network.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "network.gdb")

## Join Bike volume data to links

In [5]:
# read in links csv
links = pd.read_csv(r"..\Convert_MM_Network\Outputs\links.csv")

# read in links shapefile
links_shp = pd.DataFrame.spatial.from_featureclass(r"..\Convert_MM_Network\Outputs\links.shp")

print(links.shape)
print(links_shp.shape)

(133418, 29)
(133418, 32)


In [12]:
# read in bike volume
bike_volume = pd.read_csv(r"..\Model_Outputs\bike_vol.csv")

#should be double the amount of links for both directions
print(bike_volume.shape)

# fill bike volume NAs with -1
bike_volume['bike_vol'] = bike_volume['bike_vol'].fillna(0)


# REMOVE THIS WHEN TABLE BUG IS FIXED
bike_volume.dropna(inplace=True)

# convert node ids to int
bike_volume['from_node'] = bike_volume['from_node'].astype(int)
bike_volume['to_node'] = bike_volume['to_node'].astype(int)

(561869, 3)


In [13]:
# Create key to use for joining to links
bike_volume['key'] = np.where(bike_volume['from_node'] < bike_volume['to_node'], 
                              bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str), 
                              bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str))

# Create directional keys
bike_volume['ft_key'] = bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str)
bike_volume['tf_key'] = bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str)

bike_volume.head(15)

,from_node,to_node,bike_vol,key,ft_key,tf_key
0,18,16,4.953061e+00,16_18,18_16,16_18
1,771,769,5.486493e+01,769_771,771_769,769_771
2,786,793,2.564896e+01,786_793,786_793,793_786
3,320,347,5.817919e+00,320_347,320_347,347_320
4,917,916,5.024262e-01,916_917,917_916,916_917
5,918,2585,1.823024e-09,918_2585,918_2585,2585_918
6,991,977,1.199338e+01,977_991,991_977,977_991
7,1114,1115,2.370194e+01,1114_1115,1114_1115,1115_1114
8,1168,1169,7.033681e-01,1168_1169,1168_1169,1169_1168
9,1276,1266,1.361906e+00,1266_1276,1276_1266,1266_1276


In [14]:
# summarize trips in each direction
ft_vol_sum = pd.DataFrame(bike_volume.groupby('ft_key')['bike_vol'].sum())
tf_vol_sum = pd.DataFrame(bike_volume.groupby('tf_key')['bike_vol'].sum())

ft_vol_sum.columns = ['ft_bvol']
tf_vol_sum.columns = ['tf_bvol']

tf_vol_sum.head()

,tf_bvol
tf_key,
0_0,6.229649e+06
100000_100003,4.761698e-01
100000_100004,4.704671e+00
100000_99993,9.029862e+00
100003_100000,1.261220e+01


In [15]:
# summarize trips in both directions
volume_sum = pd.DataFrame(bike_volume.groupby('key')['bike_vol'].sum())
volume_sum.columns = ['total_bvol']
volume_sum.head(10)

,total_bvol
key,
0_0,6.229649e+06
100000_100003,1.308837e+01
100000_100004,5.823781e+00
100003_100004,9.981328e-01
100003_100023,1.417021e+01
100004_100009,5.458435e+00
100007_100013,4.639701e-01
100009_100016,5.458435e+00
100011_100014,2.679450e+01


In [16]:
#Create FTkey and TF key to use for joining to bike volumes
links['key'] = np.where(links['from_node'].astype(int) < links['to_node'].astype(int), 
                              links['from_node'].astype(str) + "_"+ links['to_node'].astype(str), 
                              links['to_node'].astype(str) + "_"+ links['from_node'].astype(str))

links[['from_node', 'to_node','key']].head(10)

,from_node,to_node,key
0,1,637,1_637
1,2,636,2_636
2,2,1,1_2
3,3,2,2_3
4,4,3,3_4
5,5,4,4_5
6,6,5,5_6
7,7,3,3_7
8,8,7,7_8
9,8,4,4_8


In [17]:
# copy the links table
links2 = links[['link_id', 'key']].copy()

# join the links with the bike volumes using the common keys
link_bike_vol = links2.merge(volume_sum, left_on='key', right_on='key', how='left')
link_bike_vol2 = link_bike_vol.merge(ft_vol_sum, left_on='key', right_on='ft_key', how='left')
link_bike_vol3 = link_bike_vol2.merge(tf_vol_sum, left_on='key', right_on='tf_key', how='left')

# examine the results
print(links2.shape)
print(link_bike_vol3.shape)

(133418, 2)
(133418, 5)


In [18]:
link_bike_vol3.head(5)

,link_id,key,total_bvol,ft_bvol,tf_bvol
0,0,1_637,NaN,NaN,NaN
1,1,2_636,NaN,NaN,NaN
2,2,1_2,0.297452,NaN,0.297452
3,3,2_3,0.484213,NaN,0.484213
4,4,3_4,0.484213,NaN,0.484213


In [19]:
# Examine the column names
links_shp.columns

Index(['FID', 'Id', 'link_id', 'temp_id', 'from_node', 'to_node', 'from_x',
       'from_y', 'to_x', 'to_y', 'Name', 'Oneway', 'Speed', 'DriveTime',
       'BikeTime', 'Pedestrian', 'Len_Miles', 'ConnectorN', 'CartoCode',
       'AADT', 'Len_Meters', 'Signal', 'Sig_Count', 'Bike_Lane', 'Bike_Path',
       'Bike_Blvd', 'from_z', 'to_z', 'Slope_AB', 'Slope_BA', 'Slope_Per',
       'SHAPE'],
      dtype='object')

In [22]:
# export final result to csv
link_bike_vol3['link_id'] = link_bike_vol3['link_id'].astype('int64')

# join bike vol to links shapefile
links4 = links_shp.merge(link_bike_vol3, left_on='link_id', right_on='link_id', how='left')

links4.fillna(0, inplace=True)


# export to shape
links4.spatial.to_featureclass(location=os.path.join(gdb,"links_bv"))

'D:\\Josh_Projects\\utah_bike_demand_model\\Post_Process_Bike_Model_Outputs\\Outputs\\network.gdb\\links_bv'

## Summarize zone trips by Attracting/Producing Zone

In [23]:
# read in zones
zones = pd.DataFrame.spatial.from_featureclass(r"..\Create_Microzones\Outputs\microzones.shp")
zones.head()

,FID,Id,zone_id,co_tazid,tazid,co_fips,co_name,residentia,households,population,jobs1,jobs3,jobs4,jobs5,jobs6,jobs7,jobs9,jobs10,jobs_total,avgincome,enrol_elem,enrol_midl,enrol_high,hhsize_lc1,hhsize_lc2,hhsize_lc3,pct_poplc1,pct_poplc2,pct_poplc3,pct_ag1,pct_ag2,pct_ag3,inc1,inc2,inc3,inc4,park_score,park_area,school_cd,coll_enrol,mtbh_score,ldr_score,th_score,comm_rail,light_rail,gqu_ratio,mixed_use,bike_share,industrial,node_id,bklane_len,bkpath_len,area_sqmil,SHAPE
0,0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,1,0,0,0.0,0.0,0,0,71531,0.754468,0.0,0.037866,"{""rings"": [[[432271.91339999996, 4511646.3509]..."
1,1,0,1,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,1,0,0,0.0,0.0,0,0,74874,2.481662,0.0,0.637239,"{""rings"": [[[432587.3509999998, 4511685.481699..."
2,2,0,2,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,1,0,0,0.0,0.0,0,0,74865,1.685400,0.0,0.573695,"{""rings"": [[[434783.8250000002, 4513368.1227],..."
3,3,0,3,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2,2,0,0,0.0,0.0,0,0,74887,1.391748,0.0,0.478397,"{""rings"": [[[436523.17530000024, 4513990.27910..."
4,4,0,4,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,2,2,0,0,0.0,0.0,0,0,74880,2.188590,0.0,0.381358,"{""rings"": [[[438170.0756000001, 4513697.879000..."


### Read in trip tables, summarize, and format

In [24]:
def summarize_zones(trips_df, name):
    
    # summarize trips by attraction or production
    trips_sum_attr = pd.DataFrame(trips_df.groupby('azone')['trips'].sum())
    trips_sum_prod = pd.DataFrame(trips_df.groupby('pzone')['trips'].sum())
    
    # format tables
    trips_sum_attr['zone_id'] = trips_sum_attr.index
    trips_sum_attr.columns = [name + '_abk', 'zone_id']
    trips_sum_prod['zone_id'] = trips_sum_prod.index
    trips_sum_prod.columns = [name + '_pbk', 'zone_id']
    
    # join the attraction and production summary tables using zone id
    merged = trips_sum_attr.merge(trips_sum_prod, left_on='zone_id', right_on='zone_id', how='outer')
    return merged
    

In [15]:
# sch_univ = pd.read_csv(r".\Data\sch_univ_trip.csv")
# sch_univ_sum = summarize_zones(sch_univ, 'univ')
# sch_univ_sum.isnull().values.any()

In [26]:
# Discretionary trips (social trips, some recreation)
disc = pd.read_csv(r"..\Model_Outputs\disc_trip.csv")
disc_sum = summarize_zones(disc, 'disc')
del disc

# Maintenance trips (e.g. groceries)
maint = pd.read_csv(r"..\Model_Outputs\maint_trip.csv")
maint_sum = summarize_zones(maint, 'mnt')
del maint

# Maintenance trips non-home-based (e.g. groceries)
maint_nhb = pd.read_csv(r"..\Model_Outputs\maint_nhb_trip.csv")
maint_nhb_sum = summarize_zones(maint_nhb, 'mntnhb')
del maint_nhb

# Recreational family trips
rec_fam = pd.read_csv(r"..\Model_Outputs\rec_fam_trip.csv")
rec_fam_sum = summarize_zones(rec_fam, 'recfam')
del rec_fam

# Recreation long trips
rec_long = pd.read_csv(r"..\Model_Outputs\rec_long_trip.csv")
rec_long_sum = summarize_zones(rec_long, 'reclng')
del rec_long

# Recreation mountain bike trips
rec_mtb = pd.read_csv(r"..\Model_Outputs\rec_mtb_trip.csv")
rec_mtb_sum = summarize_zones(rec_mtb, 'recmtb')
del rec_mtb

# Recreation other trips (recreation that doesn't fall into family or long)
rec_oth = pd.read_csv(r"..\Model_Outputs\rec_oth_trip.csv")
rec_oth_sum = summarize_zones(rec_oth, 'recoth')
del rec_oth

# school (grade) trips
sch_grade = pd.read_csv(r"..\Model_Outputs\sch_grade_trip.csv")
sch_grade_sum = summarize_zones(sch_grade, 'grade')
del sch_grade

# school (university) trips
sch_univ = pd.read_csv(r"..\Model_Outputs\sch_univ_trip.csv")
sch_univ_sum = summarize_zones(sch_univ, 'univ')
del sch_univ

# Work trips
work = pd.read_csv(r"..\Model_Outputs\work_trip.csv")
work_sum = summarize_zones(work, 'wrk')
del work

# Work non-home-based trips
work_nhb = pd.read_csv(r"..\Model_Outputs\work_nhb_trip.csv")
work_nhb_sum = summarize_zones(work_nhb, 'wrknhb')
del work_nhb

In [27]:
rec_fam_sum

,recfam_abk,zone_id,recfam_pbk
0,0.000001,158,NaN
1,0.000001,160,NaN
2,0.000004,177,NaN
3,0.000002,183,NaN
4,0.000001,199,NaN
...,...,...,...
16236,0.000316,19210,NaN
16237,0.000474,19211,NaN
16238,0.004859,19212,NaN
16239,0.002228,19213,NaN


In [28]:
zones

,FID,Id,zone_id,co_tazid,tazid,co_fips,co_name,residentia,households,population,jobs1,jobs3,jobs4,jobs5,jobs6,jobs7,jobs9,jobs10,jobs_total,avgincome,enrol_elem,enrol_midl,enrol_high,hhsize_lc1,hhsize_lc2,hhsize_lc3,pct_poplc1,pct_poplc2,pct_poplc3,pct_ag1,pct_ag2,pct_ag3,inc1,inc2,inc3,inc4,park_score,park_area,school_cd,coll_enrol,mtbh_score,ldr_score,th_score,comm_rail,light_rail,gqu_ratio,mixed_use,bike_share,industrial,node_id,bklane_len,bkpath_len,area_sqmil,SHAPE
0,0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,1,1,0,0,0.0,0.000000,0,0,71531,0.754468,0.000000,0.037866,"{""rings"": [[[432271.91339999996, 4511646.3509]..."
1,1,0,1,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,1,1,0,0,0.0,0.000000,0,0,74874,2.481662,0.000000,0.637239,"{""rings"": [[[432587.3509999998, 4511685.481699..."
2,2,0,2,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,1,1,0,0,0.0,0.000000,0,0,74865,1.685400,0.000000,0.573695,"{""rings"": [[[434783.8250000002, 4513368.1227],..."
3,3,0,3,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,2,2,0,0,0.0,0.000000,0,0,74887,1.391748,0.000000,0.478397,"{""rings"": [[[436523.17530000024, 4513990.27910..."
4,4,0,4,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,2,2,0,0,0.0,0.000000,0,0,74880,2.188590,0.000000,0.381358,"{""rings"": [[[438170.0756000001, 4513697.879000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19209,19209,0,19214,570177,177,57,WEBER,0,262,783,129,86,4,471,52,132,21,133,874,72350.640604,0.000538,0.0,0.0,1.880701,4.456423,1.859552,0.217749,0.651696,0.130554,0.316169,0.562229,0.121602,0.117415,0.189279,0.326530,0.366777,0,0,0,0,0,0,0,0,0,0.0,201.573944,0,1,99748,2.062195,0.000000,1.353920,"{""rings"": [[[414111.5, 4576365.4], [414166.599..."
19210,19210,0,19215,30131,131,3,BOX ELDER,0,1,4,0,0,0,0,0,0,0,0,0,53364.818182,0.000000,0.0,0.0,1.674617,3.826075,2.221155,0.166884,0.746178,0.086938,0.286888,0.626268,0.086844,0.226631,0.236271,0.304724,0.232373,2,122819,0,0,0,0,0,0,0,0.0,0.000000,0,0,101048,0.000000,0.000000,2.758790,"{""rings"": [[[412799.72869999986, 4580230.78240..."
19211,19211,0,19216,30122,122,3,BOX ELDER,0,0,0,0,0,0,0,0,0,0,0,0,53211.000000,0.000000,0.0,0.0,1.487271,3.322740,1.914478,0.193371,0.707086,0.099543,0.321376,0.579081,0.099543,0.235062,0.239343,0.302038,0.223557,0,0,0,0,0,0,0,0,0,0.0,0.000000,0,0,101310,0.114823,0.159936,1.822950,"{""rings"": [[[412913.78000000026, 4586186.91], ..."
19212,19212,0,19217,30059,59,3,BOX ELDER,0,0,0,0,0,0,0,0,0,0,0,0,54392.864222,0.000000,0.0,0.0,0.342818,0.772383,0.444253,0.042962,0.924093,0.032945,0.067375,0.899679,0.032945,0.368686,0.267065,0.236036,0.128213,0,0,1,0,0,0,0,0,0,0.0,0.000000,0,0,102397,0.000000,0.000000,1.716000,"{""rings"": [[[414139.03000000026, 4595085.74], ..."


## Merge trip summaries back to microzone shapefile

In [34]:
# Create a clean copy of zones dataset
# zones2 = zones[['zone_id', 'co_tazid', 'tazid', 'co_fips', 'co_name', 'SHAPE']].copy()
zones2 = zones[['zone_id', 'co_tazid', 'tazid', 'co_fips', 'co_name', 'area_sqmil', 'SHAPE']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')



# Join trip tables
zones2 = zones2.merge(disc_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(maint_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(maint_nhb_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_fam_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_long_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_mtb_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_oth_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(sch_grade_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(sch_univ_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(work_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(work_nhb_sum, left_on='zone_id', right_on='zone_id', how='left')




# preview table
zones2.tail(20)

,zone_id,co_tazid,tazid,co_fips,co_name,area_sqmil,SHAPE,disc_abk,disc_pbk,mnt_abk,mnt_pbk,mntnhb_abk,mntnhb_pbk,recfam_abk,recfam_pbk,reclng_abk,reclng_pbk,recmtb_abk,recmtb_pbk,recoth_abk,recoth_pbk,grade_abk,grade_pbk,univ_abk,univ_pbk,wrk_abk,wrk_pbk,wrknhb_abk,wrknhb_pbk
19194,19199,570264,264,57,WEBER,1.338380,"{'rings': [[[415993.6699999999, 4562007.039999...",1.566404,NaN,0.734682,NaN,2.393825e-01,0.307097,0.013502,NaN,NaN,7.137155,NaN,1.784289,1.811893e+00,NaN,7.644920,NaN,NaN,NaN,533.390002,22.888699,222.971595,406.976572
19195,19200,570313,313,57,WEBER,0.392396,"{'rings': [[[415649.3499999996, 4564583.32], [...",0.132034,NaN,0.305310,NaN,2.183470e-01,0.127620,0.038294,NaN,NaN,NaN,NaN,NaN,3.580528e-01,NaN,0.115993,NaN,NaN,NaN,35.790396,40.759284,24.470363,27.308072
19196,19201,570375,375,57,WEBER,1.373940,"{'rings': [[[422290.71389999986, 4565380.46900...",0.360227,1.751538,0.359318,NaN,1.914796e-01,0.150195,0.057404,0.012381,NaN,NaN,53.116213,NaN,8.476422e-02,4.065228,0.340612,3.165895,NaN,NaN,34.449715,116.616841,39.590479,26.285132
19197,19202,570218,218,57,WEBER,0.166881,"{'rings': [[[417534.04000000004, 4566909.10999...",0.268639,NaN,0.104123,NaN,1.733275e-01,0.043523,0.055371,NaN,NaN,NaN,NaN,NaN,3.130569e-01,NaN,0.017112,0.073981,NaN,NaN,19.977795,25.353588,14.306215,15.243058
19198,19203,570142,142,57,WEBER,14.504200,"{'rings': [[[404316.26999999955, 4567128.31000...",0.001780,NaN,NaN,NaN,3.242702e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275716,3.512209,NaN,NaN,9.370556,39.479374,7.585946,7.149734
19199,19204,570229,229,57,WEBER,0.082633,"{'rings': [[[420672.6500000004, 4567199.48], [...",0.055520,NaN,0.074307,NaN,8.695528e-03,0.031060,0.010583,NaN,NaN,NaN,NaN,NaN,1.179763e+00,NaN,0.574293,0.122377,NaN,NaN,22.200006,9.590254,21.208365,16.938604
19200,19205,570173,173,57,WEBER,0.274591,"{'rings': [[[414008.11000000034, 4568534.60999...",0.006707,NaN,0.000336,NaN,8.721100e-03,0.000140,0.018464,NaN,NaN,2.418476,NaN,0.604619,3.319851e-02,NaN,NaN,0.784167,NaN,NaN,0.734554,9.274363,0.624206,0.560465
19201,19206,570220,220,57,WEBER,0.163268,"{'rings': [[[419895.4199999999, 4567953.140000...",0.133318,NaN,0.068215,NaN,2.767165e-02,0.028514,0.081733,0.134388,NaN,NaN,NaN,NaN,2.477025e-02,NaN,0.005532,NaN,NaN,NaN,1.557913,52.381093,1.198624,1.188687
19202,19207,570214,214,57,WEBER,0.371326,"{'rings': [[[416092.3300000001, 4570986.710000...",0.029325,NaN,0.000536,NaN,6.346289e-02,0.000224,0.099310,NaN,NaN,7.969094,NaN,1.992273,2.945857e-01,NaN,0.999369,0.123768,NaN,NaN,20.106686,33.330376,13.368881,15.341401
19203,19208,570212,212,57,WEBER,0.929692,"{'rings': [[[414180.1299999999, 4571408.17], [...",0.000331,NaN,NaN,NaN,4.979742e-03,NaN,0.007078,NaN,NaN,2.472890,NaN,0.618222,8.063544e-07,NaN,1.002418,0.515587,NaN,NaN,26.472891,8.978761,14.488734,20.198816


In [35]:
zones2.columns

Index(['zone_id', 'co_tazid', 'tazid', 'co_fips', 'co_name', 'area_sqmil',
       'SHAPE', 'disc_abk', 'disc_pbk', 'mnt_abk', 'mnt_pbk', 'mntnhb_abk',
       'mntnhb_pbk', 'recfam_abk', 'recfam_pbk', 'reclng_abk', 'reclng_pbk',
       'recmtb_abk', 'recmtb_pbk', 'recoth_abk', 'recoth_pbk', 'grade_abk',
       'grade_pbk', 'univ_abk', 'univ_pbk', 'wrk_abk', 'wrk_pbk', 'wrknhb_abk',
       'wrknhb_pbk'],
      dtype='object')

In [36]:
# fill na's with 0
zones3 = zones2

# fill NAs where necessary
for field in list(zones3.columns):
    if field not in ['SHAPE']:
        zones3[field].fillna(0, inplace=True)

# calc totals        
zones3['total_abk'] = (zones3['disc_abk'] + zones3['mnt_abk'] + zones3['mntnhb_abk'] + 
                       zones3['recfam_abk'] + zones3['reclng_abk'] + zones3['recmtb_abk'] + zones3['recoth_abk'] + 
                       zones3['grade_abk'] + zones3['univ_abk'] + zones3['wrk_abk'] + zones3['wrknhb_abk']) 

zones3['total_pbk'] = (zones3['disc_pbk'] + zones3['mnt_pbk'] + zones3['mntnhb_pbk'] + 
                       zones3['recfam_pbk'] + zones3['reclng_pbk'] + zones3['recmtb_pbk'] + zones3['recoth_pbk'] + 
                       zones3['grade_pbk'] + zones3['univ_pbk'] + zones3['wrk_pbk'] + zones3['wrknhb_pbk']) 



In [37]:
# then export to shape
zones3.spatial.to_featureclass(location=os.path.join(gdb,"Microzone_Trip_Summaries"))

'D:\\Josh_Projects\\utah_bike_demand_model\\Post_Process_Bike_Model_Outputs\\Outputs\\network.gdb\\Microzone_Trip_Summaries'

## Merge zone attraction and production scores with the microzone geometry

In [41]:
# Create a clean copy of zones dataset
zones2 = zones[['zone_id', 'co_tazid', 'tazid', 'co_fips', 'co_name', 'area_sqmil', 'SHAPE']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')

# NOTE: need to add zone_id to empty field in output csv
ascore = pd.read_csv(r"..\Model_Outputs\zone_attraction_size.csv")
pscore = pd.read_csv(r"..\Model_Outputs\zone_production_size.csv")

ascore = ascore.rename(columns={"Unnamed: 0": "zone_id"})
pscore = pscore.rename(columns={"Unnamed: 0": "zone_id"})

zones3a = zones2.merge(ascore, left_on='zone_id', right_on='zone_id', how='left')
zones3p = zones2.merge(pscore, left_on='zone_id', right_on='zone_id', how='left')

In [43]:
# fill NAs where necessary
for field in list(zones3a.columns):
    if field !='SHAPE':
        zones3a[field].fillna(-1, inplace=True)

# fill NAs where necessary
for field in list(zones3p.columns):
    if field !='SHAPE':
        zones3p[field].fillna(-1, inplace=True)

zones3p.rename({'sch_grade_nhb': 'grade_nhb', 'sch_univ_nhb': 'univ_nhb', 'rec_oth_nhb':'recothnhb'}, axis=1, inplace=True)        
        
# Fill NAs with -1, then export to shape
zones3a.spatial.to_featureclass(location=os.path.join(gdb,"Microzone_A_Scores"))
zones3p.spatial.to_featureclass(location=os.path.join(gdb,"Microzone_P_Scores"))

'D:\\Josh_Projects\\utah_bike_demand_model\\Post_Process_Bike_Model_Outputs\\Outputs\\network.gdb\\Microzone_P_Scores'

## Get Centroid Nodes

In [44]:
nodes = pd.DataFrame.spatial.from_featureclass(r"..\Convert_MM_Network\Outputs\nodes.shp")
nodes['node_id'] = nodes.index
nodes.shape

(102666, 8)

In [46]:
nodes2 = nodes[['node_id', 'xcoord', 'ycoord', 'zcoord', 'SHAPE']].copy()
centroids = nodes2.merge(zones[['node_id', 'zone_id']], left_on='node_id', right_on='node_id', how='inner')
print(centroids.columns)

Index(['node_id', 'xcoord', 'ycoord', 'zcoord', 'SHAPE', 'zone_id'], dtype='object')


In [47]:
centroids = centroids[['node_id', 'xcoord', 'ycoord', 'zcoord', 'zone_id', 'SHAPE']].copy()
centroids.spatial.to_featureclass(location=os.path.join(gdb,"Microzone_Centroids"))

'D:\\Josh_Projects\\utah_bike_demand_model\\Post_Process_Bike_Model_Outputs\\Outputs\\network.gdb\\Microzone_Centroids'

## Export Map Project and Processed Data

In [48]:
# if toggle is true export a copy of the project and relevant data to a local directory
save_results = True

if save_results == True:
    
    import arcpy
    project = r'.\Bike_Model_Results.aprx'
    export_folder = r'E:\Micromobility\Bike_Model_Results'
    now = datetime.datetime.now().strftime("%d%m%Y_%H%M")
    output_file = os.path.join(export_folder, 'Bike_Model_Results_{}.ppkx'.format(now))
    arcpy.PackageProject_management(project, output_file, "EXTERNAL", extent='MINOF')

ExecuteError: Layer's data source is inaccessible: Map/Microzone_P_Scores
Layer's data source is inaccessible: Map/Microzone_A_Scores
Layer's data source is inaccessible: Map/Trips Produced
Layer's data source is inaccessible: Map/Trips Attracted
Layer's data source is inaccessible: Map/Microzone_P_Scores
Layer's data source is inaccessible: Map/Microzone_A_Scores
Layer's data source is inaccessible: Map/Trips Produced
Layer's data source is inaccessible: Map/Trips Attracted
Layer's data source is inaccessible: Map/Microzone_P_Scores
Layer's data source is inaccessible: Map/Microzone_A_Scores
Layer's data source is inaccessible: Map/Trips Produced
Layer's data source is inaccessible: Map/Trips Attracted
Layer's data source is inaccessible: Map/Microzone_P_Scores
Layer's data source is inaccessible: Map/Microzone_A_Scores
Layer's data source is inaccessible: Map/Trips Produced
Layer's data source is inaccessible: Map/Trips Attracted
Layer's data source is inaccessible: Map/Microzone_P_S